# CSV file processing

In this notebook I'll test the macro, written in python, used for converting a ROOT structure (Tree) into a CSV file for ML analysis.

The CSV is located inside the directory output, created by the python macro **examplemacro.py**:

In [1]:
!ls ../MuonPOGAnalysisTemplate/

Jupyter_Notebooks_miscellaneous exampleMacro.C
MuonTree.root                   exampleMacro.py
Pt.pdf                          interface
README.md                       oneOverPt.pdf
SingleMu22_pt_turn-on.pdf       output
analysisMacro.py                qOverPt.pdf
config                          results
exampleMacro                    tdrstyle.C


In [2]:
%cd ../MuonPOGAnalysisTemplate/output
!head -5 output_muons.csv

/Users/tommaso/TESI_MAGISTRALE/MuonPOGAnalysisTemplate/output







To analyze it, we need the ROOT python module and pandas for the CSV analysis:

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import progressbar

Now we need to read the CSV:

In [4]:
df2 = pd.read_csv('./output_muons.csv')
df2

,Event,dtPrimitive.id_r,dtPrimitive.id_eta,dtPrimitive.id_phi,dtPrimitive.phiGlb(),dtPrimitive.phiB,dtPrimitive.quality,l1Muon.eta,l1Muon.phi,l1Muon.charge,genParticle.status,l1Muon.quality,l1Muon.pt,genParticle.pdgId,genParticle.eta,genParticle.phi,genParticle.pt
0,1,1,0,9,4.086740,-4.0,6,0.054375,-2.203593,1,1,12,139.5,-13,0.056219,-2.180867,148.023346
1,1,2,0,9,4.085275,-3.0,6,0.054375,-2.203593,1,1,12,139.5,-13,0.056219,-2.180867,148.023346
2,1,3,0,9,4.084298,-3.0,6,0.054375,-2.203593,1,1,12,139.5,-13,0.056219,-2.180867,148.023346
3,1,4,0,9,4.083566,-1.0,6,0.054375,-2.203593,1,1,12,139.5,-13,0.056219,-2.180867,148.023346
4,2,2,2,12,5.677555,0.0,6,0.793875,-0.611025,1,1,12,139.5,-13,0.823954,-0.591595,154.152847
5,2,3,2,12,5.676823,-3.0,6,0.793875,-0.611025,1,1,12,139.5,-13,0.823954,-0.591595,154.152847
6,3,1,1,11,5.128810,-28.0,6,0.369750,-1.167333,1,1,12,29.0,-13,0.376618,-1.061424,24.931395
7,3,2,1,11,5.120753,-20.0,5,0.369750,-1.167333,1,1,12,29.0,-13,0.376618,-1.061424,24.931395
8,3,4,1,11,5.114161,-3.0,6,0.369750,-1.167333,1,1,12,29.0,-13,0.376618,-1.061424,24.931395
9,4,1,0,3,1.175616,-6.0,6,0.000000,1.167156,1,1,12,139.5,-13,0.018205,1.192800,133.895721


We sort the dataframe with ascending order of sector primitive

In [ ]:
df2 = df2.sort_values(["Event","dtPrimitive.id_r"])
df2 = df2.reset_index(drop=True)
df2
#df2["dtPrimitive.phiB"] = df2["dtPrimitive.phiGlb()"] + df2["dtPrimitive.phiB"]/512.

### Change CSV structure for a suitable ML format

Using the following code, the structure of the input CSV changes. Instead of considering each line as a single primitive, now each line represents a single event (with multiple primitives structured by columns).

The main difference, therefore, is an increasing number of columns: one for each primitive.

In [5]:
bar = progressbar.ProgressBar(maxval=len(df2.index), widgets = [progressbar.Bar('=','[',']'), '', progressbar.Percentage()])
bar.start()
a = 1
temp = 0
new_column = []
row_list=[]
final_row=[]
for column in df2.columns.values.tolist():
    if column==df2.columns.values.tolist()[0]:
        new_column.append(column)
        new_column.append("n_Primitive")
        continue
    for count in range(1,5):
        if "()" in column:
            column = column.replace("()", "")
        new_column.append(str(count) + column)
df = pd.DataFrame(columns=new_column)
for index, row in df2.iterrows():
    if (row["Event"] == a):
        if row["dtPrimitive.id_r"] > temp:
            row_list.append(row.tolist())
            temp = row["dtPrimitive.id_r"]
            continue
        del row_list[:]
        temp = 999
        continue
    elif(row["Event"] != a):
        for i in range(0,len(df2.columns)):
            if not row_list:
                continue
            if i==0:
                final_row.append(row_list[0][0])
                final_row.append(len(row_list))
                continue
            for j in range(0,len(row_list)):
                if len(row_list)==4:
                    final_row.append(row_list[j][i])
                else:
                    final_row.append(row_list[j][i])
                    if j==len(row_list)-1:
                        final_row += [np.nan]*(4-len(row_list))
        a = row["Event"]
        if len(row_list) <=4:
            if final_row:
                df.loc[row["Event"]-1] = final_row
        del row_list[:]
        del final_row[:]
        temp = 0
        row_list.append(row.tolist())
    bar.update(index+1)

for i in range(0,len(df2.columns)):
    if not row_list:
        continue
    if i==0:
        final_row.append(row_list[0][0])
        final_row.append(len(row_list))
        continue
    for j in range(0,len(row_list)):
        if len(row_list)==4:
            final_row.append(row_list[j][i])
        else:
            final_row.append(row_list[j][i])
            if j==len(row_list)-1:
                final_row += [np.nan]*(4-len(row_list))
if len(row_list) <=4:
    if final_row:
        df.loc[row["Event"]] = final_row
bar.finish()

[=========================================================================]100%


Next it is important to remove the column that aren't essential such as genParticle duplicate

In [6]:
df = df.rename(columns = {"1genParticle.pt":"genParticle.pt","1genParticle.phi":"genParticle.phi","1genParticle.eta":"genParticle.eta","1genParticle.pdgId":"genParticle.pdgId","1genParticle.status":"genParticle.status"})

In [ ]:
for column in df.columns.values.tolist():
    if column[0] == '5':
        df = df.drop(column,axis=1)

In [7]:
for i in range(2,5):
    title = str(i) + "genParticle.pt"
    df = df.drop(title,1)

In [8]:
for i in range(2,5):
    title = str(i) + "genParticle.eta"
    df = df.drop(title,1)

In [9]:
for i in range(2,5):
    title = str(i) + "genParticle.phi"
    df = df.drop(title,1)

In [10]:
for i in range(2,5):
    title = str(i) + "genParticle.pdgId"
    df = df.drop(title,1)

In [11]:
for i in range(2,5):
    title = str(i) + "genParticle.status"
    df = df.drop(title,1)

Insert columns relative to delta phi angles between sectors.

In [23]:
cols = df.columns.values.tolist()
df.insert(len(cols)-1,"delta_phi34", np.arccos(np.cos(abs(df["3dtPrimitive.phiGlb"]-df["4dtPrimitive.phiGlb"]))))
df.insert(len(cols)-1,"delta_phi24", np.arccos(np.cos(abs(df["2dtPrimitive.phiGlb"]-df["4dtPrimitive.phiGlb"]))))
df.insert(len(cols)-1,"delta_phi23", np.arccos(np.cos(abs(df["2dtPrimitive.phiGlb"]-df["3dtPrimitive.phiGlb"]))))
df.insert(len(cols)-1,"delta_phi14", np.arccos(np.cos(abs(df["1dtPrimitive.phiGlb"]-df["4dtPrimitive.phiGlb"]))))
df.insert(len(cols)-1,"delta_phi13", np.arccos(np.cos(abs(df["1dtPrimitive.phiGlb"]-df["3dtPrimitive.phiGlb"]))))
df.insert(len(cols)-1,"delta_phi12", np.arccos(np.cos(abs(df["1dtPrimitive.phiGlb"]-df["2dtPrimitive.phiGlb"]))))

In [24]:
for index, row in df.iterrows():
    if row["1dtPrimitive.phiGlb"]==0:
        row["delta_phi13"] = 0
        row["delta_phi14"] = 0
        row["delta_phi12"] = 0
    if row["2dtPrimitive.phiGlb"]==0:
        row["delta_phi23"] = 0
        row["delta_phi24"] = 0
        row["delta_phi12"] = 0
    if row["3dtPrimitive.phiGlb"]==0:
        row["delta_phi23"] = 0
        row["delta_phi34"] = 0
        row["delta_phi13"] = 0
    if row["4dtPrimitive.phiGlb"]==0:
        row["delta_phi34"] = 0
        row["delta_phi24"] = 0
        row["delta_phi14"] = 0
    

In [25]:
for column in df.columns.values.tolist():
    if "phiGlb" in column:
        df = df.drop(column,axis=1)

Let's plot a correlation matrix for each variable inside the csv:

In [ ]:
def plot_corr(df,size=10):
    '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''

    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns,rotation=90);
    plt.yticks(range(len(corr.columns)), corr.columns);
    plt.imshow(corr, aspect=1)

In [ ]:
plot_corr(df)
plt.colorbar()
plt.show()

Then put the organized table inside a csv file.

In [27]:
df = df[df.n_Primitive != 1.0]

In [52]:
df

,Event,n_Primitive,1dtPrimitive.id_r,2dtPrimitive.id_r,3dtPrimitive.id_r,4dtPrimitive.id_r,1dtPrimitive.id_eta,2dtPrimitive.id_eta,3dtPrimitive.id_eta,4dtPrimitive.id_eta,1dtPrimitive.id_phi,2dtPrimitive.id_phi,3dtPrimitive.id_phi,4dtPrimitive.id_phi,1dtPrimitive.phiB,2dtPrimitive.phiB,3dtPrimitive.phiB,4dtPrimitive.phiB,1dtPrimitive.quality,2dtPrimitive.quality,3dtPrimitive.quality,4dtPrimitive.quality,1l1Muon.eta,2l1Muon.eta,3l1Muon.eta,4l1Muon.eta,1l1Muon.phi,2l1Muon.phi,3l1Muon.phi,4l1Muon.phi,1l1Muon.charge,2l1Muon.charge,3l1Muon.charge,4l1Muon.charge,genParticle.status,1l1Muon.quality,2l1Muon.quality,3l1Muon.quality,4l1Muon.quality,1l1Muon.pt,2l1Muon.pt,3l1Muon.pt,4l1Muon.pt,genParticle.pdgId,genParticle.eta,genParticle.phi,delta_phi12,delta_phi13,delta_phi14,delta_phi23,delta_phi24,delta_phi34,genParticle.pt
1.0,1.0,4.0,1.0,2.0,3.0,4.0,0.0,0.0,0.0,0.0,9.0,9.0,9.0,9.0,-4.0,-3.0,-3.0,-1.0,6.0,6.0,6.0,6.0,0.054375,0.054375,0.054375,0.054375,-2.203593,-2.203593,-2.203593,-2.203593,1.0,1.0,1.0,1.0,1.0,12.0,12.0,12.0,12.0,139.5,139.5,139.5,139.5,-13.0,0.056219,-2.180867,0.001465,0.002441,0.003174,0.000977,0.001709,0.000732,148.023346
2.0,2.0,2.0,2.0,3.0,NaN,NaN,2.0,2.0,NaN,NaN,12.0,12.0,NaN,NaN,0.0,-3.0,NaN,NaN,6.0,6.0,NaN,NaN,0.793875,0.793875,NaN,NaN,-0.611025,-0.611025,NaN,NaN,1.0,1.0,NaN,NaN,1.0,12.0,12.0,NaN,NaN,139.5,139.5,NaN,NaN,-13.0,0.823954,-0.591595,0.000732,NaN,NaN,NaN,NaN,NaN,154.152847
3.0,3.0,3.0,1.0,2.0,4.0,NaN,1.0,1.0,1.0,NaN,11.0,11.0,11.0,NaN,-28.0,-20.0,-3.0,NaN,6.0,5.0,6.0,NaN,0.369750,0.369750,0.369750,NaN,-1.167333,-1.167333,-1.167333,NaN,1.0,1.0,1.0,NaN,1.0,12.0,12.0,12.0,NaN,29.0,29.0,29.0,NaN,-13.0,0.376618,-1.061424,0.008057,0.014648,NaN,0.006592,NaN,NaN,24.931395
8.0,8.0,2.0,1.0,4.0,NaN,NaN,1.0,1.0,NaN,NaN,2.0,2.0,NaN,NaN,-3.0,1.0,NaN,NaN,6.0,6.0,NaN,NaN,0.369750,0.369750,NaN,NaN,0.589032,0.589032,NaN,NaN,1.0,1.0,NaN,NaN,1.0,12.0,12.0,NaN,NaN,139.5,139.5,NaN,NaN,-13.0,0.355098,0.607802,0.002197,NaN,NaN,NaN,NaN,NaN,199.715729
9.0,9.0,4.0,1.0,2.0,3.0,4.0,1.0,1.0,1.0,1.0,7.0,7.0,7.0,7.0,-5.0,-11.0,-4.0,1.0,6.0,3.0,6.0,6.0,0.413250,0.413250,0.413250,0.413250,-2.978061,-2.978061,-2.978061,-2.978061,1.0,1.0,1.0,1.0,1.0,12.0,12.0,12.0,12.0,139.5,139.5,139.5,139.5,-13.0,0.410197,-2.953913,0.001465,0.002197,0.002441,0.000732,0.000977,0.000244,146.011047
10.0,10.0,2.0,1.0,2.0,NaN,NaN,-2.0,-2.0,NaN,NaN,7.0,7.0,NaN,NaN,-4.0,-1.0,NaN,NaN,6.0,5.0,NaN,NaN,-0.978750,-0.978750,NaN,NaN,3.021516,3.021516,NaN,NaN,1.0,1.0,NaN,NaN,1.0,12.0,12.0,NaN,NaN,140.0,140.0,NaN,NaN,-13.0,-0.977629,3.052300,0.001221,NaN,NaN,NaN,NaN,NaN,127.335487
11.0,11.0,4.0,1.0,2.0,3.0,4.0,1.0,1.0,1.0,1.0,8.0,8.0,8.0,8.0,-15.0,-13.0,-9.0,51.0,6.0,5.0,6.0,3.0,0.369750,0.369750,0.369750,0.369750,-2.672637,-2.672637,-2.672637,-2.672637,1.0,1.0,1.0,1.0,1.0,12.0,12.0,12.0,12.0,45.5,45.5,45.5,45.5,-13.0,0.353162,-2.613130,0.005127,0.009033,0.004639,0.003906,0.000488,0.004395,48.427322
12.0,12.0,2.0,1.0,2.0,NaN,NaN,-2.0,-2.0,NaN,NaN,8.0,8.0,NaN,NaN,-9.0,3.0,NaN,NaN,6.0,2.0,NaN,NaN,-1.076625,-1.076625,NaN,NaN,-2.585373,-2.585373,NaN,NaN,1.0,1.0,NaN,NaN,1.0,12.0,12.0,NaN,NaN,80.0,80.0,NaN,NaN,-13.0,-1.073775,-2.545777,0.001221,NaN,NaN,NaN,NaN,NaN,90.242088
15.0,15.0,2.0,1.0,2.0,NaN,NaN,-2.0,-2.0,NaN,NaN,6.0,7.0,NaN,NaN,-40.0,-27.0,NaN,NaN,5.0,6.0,NaN,NaN,-0.924375,-0.924375,NaN,NaN,2.912436,2.912436,NaN,NaN,1.0,1.0,NaN,NaN,1.0,12.0,12.0,NaN,NaN,18.0,18.0,NaN,NaN,-13.0,-0.939358,3.072543,0.009604,NaN,NaN,NaN,NaN,NaN,15.613245
16.0,16.0,3.0,1.0,2.0,4.0,NaN,0.0,0.0,1.0,NaN,11.0,12.0,12.0,NaN,-41.0,-15.0,-7.0,NaN,4.0,2.0,6.0,NaN,0.228375,0.228375,0.228375,NaN,-0.796461,-0.796461,-0.796461,NaN,1.0,1.0,1.0,NaN,1.0,12.0,12.0,12.0,NaN,22.0,22.0,22.0,NaN,-13.0,0.208122,-0.632215,0.010337,0.022544,NaN,0.012207,NaN,NaN,17.377583


In [43]:
df.to_csv("/Users/tommaso/TESI_MAGISTRALE/MuonPOGAnalysisTemplate/output/bxcut_full_4.csv",na_rep=0,index=False)